# Check if tracer has saturated the domain and how other species are changing with time

In [1]:
import os
import pandas as pd
import numpy as np

#Developed libraries for working with simulation results
import plots.saturated_steady_state as sssp
import analyses.transient as translys
from data_reader import data_processing as proc

#Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

## Parent directories

In [2]:
results_dir = "Y:/Home\khurana/4. Publications/Paper3/Figurecodes"
#output_dir = "Y:/Home\khurana/4. Publications/Paper3/Figurecodes"
raw_dir = "X:/Richards_flow_big_sat"#"X:/Richards_flow_big_sat"

In [3]:
Regimes = ["Fast", "Medium","Slow"]
trialist = proc.masterscenarios("Unsaturated")
mTrial = list(trialist.keys())
droplist = []
Trial = list(t for t in mTrial if t not in droplist)
def search_variable_indict(my_dict, value_of_tec_index):
    for t in my_dict.keys():
        i = species[t]['TecIndex']
        if i==value_of_tec_index:
            name = t
            break
        else:
            name = 'NA'
    return name

## Check for average, minimum, maximum percentage difference in values between two time points for steady state
### Each simulation is running for 400 days currently

In [4]:
species = proc.speciesdict("Unsaturated")
initial = 1
final = -1
yin = 6
yout = -6
row = []
for r in Regimes:
    for t in Trial:
        file = os.path.join(raw_dir, r+"AR_0", r+ "AR_0_RF-A"+str(t)+"_df.npy")
        data = np.load(file)
        if np.shape(data)[1]>1:
            for v in list(range(np.shape(data)[0])):
                percentage_diff = abs(100*(data[v,final,yin:yout,:] - data[v,initial,yin:yout,:])/data[v,initial,yin:yout,:])
                median_diff = np.median(percentage_diff)
                average_diff = np.mean(percentage_diff)
                varname = search_variable_indict(species,v)
                row.append([r,t,varname,median_diff, average_diff])
        else:
            continue

<ipython-input-4-beb2218ecf8a>:13: RuntimeWarning: invalid value encountered in true_divide
  percentage_diff = abs(100*(data[v,final,yin:yout,:] - data[v,initial,yin:yout,:])/data[v,initial,yin:yout,:])


In [5]:
df = pd.DataFrame.from_records(row, columns=["Regime","Trial","Feature","Median_diff","Average_diff"])
df.to_csv(os.path.join(results_dir,"check_steady_state_21062021.csv"))

In [6]:
df.groupby(["Regime","Feature"])["Average_diff"].describe()

count          mean           std  \
Regime Feature                                                                
Fast   Ammonium                            49.0  1.017304e-02  6.123114e-02   
       DO                                  49.0  1.036111e-04  4.208140e-04   
       DOC                                 49.0  2.816552e-04  1.236759e-03   
       Immobile active aerobic degraders   49.0  5.661861e-03  2.357279e-02   
       Immobile active ammonia oxidizers   49.0  3.634293e-02  1.703577e-01   
...                                         ...           ...           ...   
Slow   NA                                 196.0  3.092809e-01  4.313337e+00   
       Nitrate                             49.0  1.052216e-03  4.054082e-03   
       Particulate organic matter          49.0  3.835749e+00  2.516539e+01   
       Sulphate                            49.0  9.138252e-12  5.702547e-11   
       Tracer                              49.0  3.986134e-09  2.537971e-08   

                                                   min           25%  \
Regime Feature                                                         
Fast   Ammonium                           9.325529e-10  1.165440e-09   
       DO                                 1.559821e-08  2.460708e-08   
       DOC                                2.853476e-09  3.906031e-09   
       Immobile active aerobic degraders  5.722177e-09  6.954221e-09   
       Immobile active ammonia oxidizers  5.301752e-08  5.387369e-08   
...                                                ...           ...   
Slow   NA                                 2.648993e-13  7.161155e-12   
       Nitrate                            1.759566e-09  8.730944e-08   
       Particulate organic matter         1.694026e-07  2.238543e-07   
       Sulphate                           1.082064e-14  7.574694e-14   
       Tracer                             0.000000e+00  0.000000e+00   

                                                   50%           75%  \
Regime Feature                                                         
Fast   Ammonium                           4.319334e-09  9.668211e-09   
       DO                                 7.616498e-08  1.934949e-07   
       DOC                                2.366989e-08  5.060170e-08   
       Immobile active aerobic degraders  3.797454e-08  7.432973e-08   
       Immobile active ammonia oxidizers  1.170755e-07  2.346318e-07   
...                                                ...           ...   
Slow   NA                                 9.638206e-07  9.845034e-05   
       Nitrate                            1.102115e-07  3.437712e-06   
       Particulate organic matter         7.620210e-03  2.114481e-01   
       Sulphate                           1.082064e-13  2.488747e-13   
       Tracer                             0.000000e+00  0.000000e+00   

                                                   max  
Regime Feature                                          
Fast   Ammonium                           4.271852e-01  
       DO                                 2.605247e-03  
       DOC                                7.748913e-03  
       Immobile active aerobic degraders  1.448916e-01  
       Immobile active ammonia oxidizers  1.108961e+00  
...                                                ...  
Slow   NA                                 6.038759e+01  
       Nitrate                            2.079771e-02  
       Particulate organic matter         1.763616e+02  
       Sulphate                           3.993514e-10  
       Tracer                             1.771014e-07  

[72 rows x 8 columns]

In [34]:
df.groupby(["Regime","Trial"])["Average_diff"].describe()

count          mean           std           min           25%  \
Regime Trial                                                                  
Fast   107     27.0  2.763735e-07  5.553942e-07  0.000000e+00  3.734374e-08   
       111     27.0  7.943388e+00  4.021520e+01  0.000000e+00  4.673874e-03   
       114     27.0  2.937328e-07  9.340318e-07  0.000000e+00  1.754335e-08   
       116     27.0  1.121455e-06  4.745117e-06  0.000000e+00  5.612430e-08   
       118     27.0  6.279743e-04  1.125488e-03  0.000000e+00  3.789688e-05   
...             ...           ...           ...           ...           ...   
Slow   84      27.0  7.109563e-01  9.111709e-01  0.000000e+00  2.697595e-03   
       89      27.0  1.877331e+18  7.770463e+18  1.471979e-11  2.963493e-02   
       90      27.0  6.702807e+09  2.463231e+10  6.889186e-13  1.300132e-02   
       98      27.0  7.810345e+06  3.059983e+07  0.000000e+00  3.670249e-02   
       H       27.0  4.415180e-01  2.294191e+00  0.000000e+00  7.844582e-08   

                       50%           75%           max  
Regime Trial                                            
Fast   107    7.123540e-08  1.965503e-07  2.816528e-06  
       111    2.288950e-01  3.611603e-01  2.091660e+02  
       114    5.465843e-08  2.154786e-07  4.908641e-06  
       116    1.616706e-07  2.976304e-07  2.483397e-05  
       118    1.423447e-04  3.911600e-04  4.034249e-03  
...                    ...           ...           ...  
Slow   84     1.899808e-01  1.913018e+00  2.188408e+00  
       89     5.939041e+00  2.217572e+03  3.999921e+19  
       90     1.945460e+00  3.154708e+00  1.079871e+11  
       98     2.258457e+00  6.075811e+00  1.487819e+08  
       H      2.814134e-07  9.824562e-07  1.192097e+01  

[147 rows x 8 columns]

In [7]:
df[(df.Regime=="Medium") & (df.Trial=="H")]

,Regime,Trial,Feature,Median_diff,Average_diff
1372,Medium,H,NA,0.000000e+00,4.281835e-12
1373,Medium,H,NA,3.935577e-02,8.218126e+00
1374,Medium,H,NA,6.978854e-07,8.092716e-07
1375,Medium,H,NA,NaN,NaN
1376,Medium,H,NA,0.000000e+00,9.174437e-13
1377,Medium,H,Immobile active aerobic degraders,1.088769e-07,1.449027e-07
1378,Medium,H,Mobile active aerobic degraders,8.982282e-08,1.311732e-07
1379,Medium,H,DOC,1.828956e-07,1.618993e-07
1380,Medium,H,DO,2.107308e-07,2.000845e-07
1381,Medium,H,Ammonium,1.705759e-08,1.755653e-08


## Check for incoming water velocity and outgoing water velocity at the last time step

In [ ]:
gvarnames = ["P","VX", "VY", "Sat", "Tracer"]
gindx = [0,1,2,4]
velindex = 2
vedge = 0.01/2
velem = 0.01
por = 0.2
yin = [0,4,5,6]
yout = [-1, -4, -6]
voly=0.005 #0.01
def effsat(data):
    slope = 1/(0.8-0.2)
    constant = -1/3
    sat = slope*data + constant
    return sat
row = []
for yi in yin:
    for yo in yout:
        for r in Regimes:
            for t in Trial:
                file = os.path.join(raw_dir, r+"AR_0", r+ "AR_0_RF-A"+str(t)+"_df.npy")
                data = np.load(file)
                velx = data[1, -1, :, :]
                vely = data[2, -1, :, :]
                sat = data[4,-1,:,:]
                veledgi = (vely[yi,0]*sat[yi,0] + vely[yi,-1]*sat[yi,-1])*0.005*voly
                veledgo = (vely[yo,-1]*sat[yo,-1] + vely[yo,-1]*sat[yo,-1])*0.005*voly
                sumvelin = sum(vely[yi,1:-1]*sat[yi,1:-1]+vely[yi,1:-1]*sat[yi,1:-1])*voly*0.01 + veledgi
                sumvelout = sum(vely[yo,1:-1]*sat[yo,1:-1]+vely[yo,1:-1]*sat[yo,1:-1])*voly*0.01 + veledgo
                #sumvelin = sum(vely[yi,:]*sat[yi,:])
                #sumvelout = sum(vely[yo,:]*sat[yo,:])
                delta = sumvelin-sumvelout
                row.append([r,t,trialist[t]["Het"],trialist[t]["Anis"],sumvelin, sumvelout, delta, delta/sumvelin, veledgi, veledgo, yi, yo])

In [ ]:
in_outvel = pd.DataFrame.from_records(row,columns = ["Regime", "Trial", "Variance", "Anisotropy","flux_in","flux_out","Delta", "Rel_delta", "flux_edge_in", "flux_edge_out", "Inlet", "Outlet"])    
in_outvel.to_csv(os.path.join(results_dir,"in_out_flux_comparison_bigsat_07062021.csv"))

In [ ]:
in_outvel.groupby(["Regime", "Variance"])["Rel_delta"].median()*100

In [ ]:
Regimes = ["Slow", "Equal", "Fast"]
trialist = proc.masterscenarios()
Trial = list(trialist.keys())
species = proc.speciesdict("Unsaturated")
gvarnames = ["P","VX", "VY", "Sat", "Tracer"]
gindx = [0,1,2,4]
velindex = 2
vedge = 0.01/2
velem = 0.01
por = 0.2
def effsat(data):
    slope = 1/(0.8-0.2)
    constant = -1/3
    sat = slope*data + constant
    return sat
row = []
for r in Regimes:
    for t in Trial:
        file = os.path.join(raw_dir, r+"AR_0", r+ "AR_0_RF-A"+str(t)+"_df.npy")
        data = np.load(file)
        velx = data[1, -1, :, :]
        vely = data[2, -1, :, :]
        sat = data[4,-1,:,:]
        velledg = vely[0,0]
        velredg = vely[0,-1]
        velelem = sum(vely[0,1:-1])/29
        velin = np.sum(velledg+velredg + velelem*29)/31
        sumvelelemin = np.sum(velelem)
        sumvelin = np.sum(velelem)+velledg+velredg
        row.append([r,t,trialist[t]["Het"],trialist[t]["Anis"],velledg, velredg, velelem, velin, sumvelelemin, sumvelin])

In [ ]:
veldata = pd.DataFrame.from_records(row,columns = ["Regime", "Trial", "Variance", "Anisotropy","Vel_left_edge", "Vel_right_edge", "Avg_Vel_elem","Avg_Vel_in", "Sum_Vel_elements", "Sum_Vel_in"])
veldata.to_csv(os.path.join(results_dir,"Vel_comparison_06042021.csv"))